## Data Import and Preprocessing

In [ ]:
#Import necessary libraries
import pandas as pd
import numpy as np

In [ ]:
#import data
i8 = pd.read_excel(r"C:\Users\shrut\OneDrive\Desktop\Training\B-Score\data\i8_cards 2.xlsx")
#i1 = pd.read_excel(r"C:\Users\shrut\OneDrive\Desktop\Training\B-Score\data\i1_card_transaction 2.xlsx")

In [ ]:
data=i8
#data = pd.merge(i8, i1, how='left', on='MASK_CREDITCARD')
data

In [ ]:
#data has 142417 rows and 26 columns
data.shape

In [ ]:
#change datatype of certain columns to desired datatypes
data['NB_DAYS_DELINQ'] = data['NB_DAYS_DELINQ'].astype(str)
data['REGIONS'] = data['REGIONS'].astype(str)
data['NATIONALITY'] = data['NATIONALITY'].astype(str)
data.dtypes

In [ ]:
#examine data
data.head()

In [ ]:
#sort data using MASK_CREDITCARD. Within each MASK_CREDITCARD further dort using PERIOD 
data=data.sort_values(['MASK_CREDITCARD','PERIOD'])

### Continuous Reporting

Check whether the reporting dates for all accounts are continuous.

In [ ]:
#create a list for missing reportings. 
#check which accounts have missing reportings by compariing expected dates and actual dates
#remove accounts having discontinuous reporting

missing_reporting=[]
for account, group in data.groupby('MASK_CREDITCARD'):
    actual_months = group['PERIOD'].dt.to_period('M').unique()
    expected_months = pd.period_range(start=group['PERIOD'].min(), end=group['PERIOD'].max(), freq='M')
    missing_months = np.setdiff1d(expected_months, actual_months)
    if missing_months.size > 0:
        missing_reporting.extend([(account, month) for month in missing_months])
        #print(missing_reporting)
        #print(f"Missing reporting months for Credit Card ID {account} are : {missing_months}")

for account, month in missing_reporting:
    data = data[~(data['MASK_CREDITCARD'] == account)]
print(f"No of unique credit cards : {len(data['MASK_CREDITCARD'].unique())}")
print(f"No of credit cards with missing reporting : {len(missing_reporting)}")
print(f"Credit cards with missing reportings : ")
missing_reporting_df = pd.DataFrame(missing_reporting, columns=['MASK_CREDITCARD', 'MISSING_MONTHS'])
missing_reporting_df

In [ ]:
data.shape #examine shape of new dataset

### Missing Data Imputation

In [ ]:
data.isnull().sum() #check the no of missing values in each column

In [ ]:
#impute missing values in numerical variables with 0 
#impute missing values in categorical variables with 'MISSING'
#check if any more missing values are being identified

numeric_cols = data.select_dtypes(include=['number']).columns
categorical_cols = data.select_dtypes(include=['object', 'category']).columns
print(numeric_cols)
print(categorical_cols)
data[numeric_cols] = data[numeric_cols].fillna(0)
data[categorical_cols] = data[categorical_cols].fillna('MISSING')
data.isnull().sum()

### Duplicates

In [ ]:
#check if there are any duplicates in the data
duplicates = data[data.duplicated()==True]
print(duplicates)

In [ ]:
#drop duplicated rows from data
data = data.drop_duplicates()
data

### Impute Date Columns

In [ ]:
#in some cases, reporting date and other dates are in middle of the month. substitute them with dates at end of same month
date_columns = ['PERIOD','a_issue_date2','Customer_Join_Date']
for col in date_columns:
    data[col] = data[col] + pd.offsets.MonthEnd(0)
data

### Same DPD for same PERIOD and MASK_CREDITCARD 

In [ ]:
#check if there are any rows with same MASK_CREDITCARD and same PERIOD which have different DPD Days.
#in that case, impute them with max DPD and check once again.

duplicate_groups = data.groupby(['PERIOD', 'MASK_CREDITCARD']).filter(lambda x: len(x) > 1)

print("Rows with the same PERIOD and MASK_CREDITCARD before transformation:")
print(duplicate_groups['NB_DAYS_DELINQ'])

In [ ]:
data['NB_DAYS_DELINQ'] = data.groupby(['PERIOD', 'MASK_CREDITCARD'])['NB_DAYS_DELINQ'].transform('max')

In [ ]:
duplicate_groups = data.groupby(['PERIOD', 'MASK_CREDITCARD']).filter(lambda x: len(x) > 1)

print("Rows with the same PERIOD and MASK_CREDITCARD after transformation:")
duplicate_groups['NB_DAYS_DELINQ']

### Replace values in NB_DAYS_DELINQ that are greater than 4 with '5+'

In [ ]:
data['NB_DAYS_DELINQ'] = data['NB_DAYS_DELINQ'].replace('nan', '0.0').apply(lambda x: '5.0' if (pd.notna(x) and float(x) > 4) else x)
data['NB_DAYS_DELINQ'].unique()

In [ ]:
data['NB_DAYS_DELINQ'].value_counts()

In [ ]:
data.drop(index=101839,axis=0,inplace=True)

In [ ]:
data1 = data.copy()

## Roll Rate

In [ ]:
#find credit card accounts having atleast 18 months of reporting data. drop accounts which have less than 18  months of data

data1['entry_count'] = data1.groupby('MASK_CREDITCARD')['MASK_CREDITCARD'].transform('size')
data1 = data1[data1['entry_count'] >= 18].drop(['entry_count'],axis=1)
len(data1['MASK_CREDITCARD'].unique())

In [ ]:
data1

In [ ]:
#sort this new data by MASK_CREDITCARD and then further by PERIOD. 
# divide the data into data from first 9 months and data from 9 to 18 months

data1 = data1.sort_values(['MASK_CREDITCARD','PERIOD'])
first_9_months = data1.groupby('MASK_CREDITCARD').head(9)
next_9_months = data1.groupby('MASK_CREDITCARD').apply(lambda x: x.iloc[9:18]).reset_index(drop=True)

#len(next_9_months['MASK_CREDITCARD'].unique())
first_9_months

In [ ]:
#create a transition matrix representing the no of accounts that show change in DPD Bucket

transition_matrix = pd.crosstab(
    first_9_months.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].max(),
    next_9_months.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].max(),
    rownames=['First 9 months'],
    colnames=['Next 9 months']
)
transition_matrix

In [ ]:
#calculate rolling forward rate and rolling backward rate

buckets = ['0.0','1.0','2.0','3.0','4.0','5.0']
roll_forward_rates = []
for bucket in buckets:
    total = transition_matrix.loc[bucket].sum()
    forward = transition_matrix.loc[bucket, buckets[buckets.index(bucket)+1:]].sum()
    roll_forward_rates.append((forward / total) * 100)
#print(roll_forward_rates)

buckets = ['0.0','1.0','2.0','3.0','4.0','5.0']
roll_backward_rates = []
for bucket in buckets:
    total = transition_matrix.loc[bucket].sum()
    backward = transition_matrix.loc[bucket,buckets[:buckets.index(bucket)]].sum()
    roll_backward_rates.append((backward/ total) * 100)
#print(roll_backward_rates)

roll_rate = pd.DataFrame({'buckets' : buckets, 'roll_forward_rates' : roll_forward_rates, 'roll_backward_rates' : roll_backward_rates})
roll_rate

## Vintage Analysis

In [ ]:
data2 = data.copy()
data2

In [ ]:
data2['First_Reporting'] = data2.groupby('MASK_CREDITCARD')['PERIOD'].transform('min')
data2['Default'] = data2['NB_DAYS_DELINQ'].astype(float) >= 4
data2['First_Default_Date'] = data2.groupby('MASK_CREDITCARD')['PERIOD'].transform(lambda x: x[data2['Default']].min())
data2['Is_First_Default'] = data2['PERIOD'] == data2['First_Default_Date']
data2['Months_to_Default'] = np.where(
    data2['Is_First_Default'],
    (data2['PERIOD'] - data2['First_Reporting']).dt.days // 30,
    np.nan
)
data2['First_Reporting'] = data2.groupby('MASK_CREDITCARD')['PERIOD'].transform('min').dt.year
data2

In [ ]:
pivot_table = pd.pivot_table(
    data2.dropna(subset=['Months_to_Default']),
    values='MASK_CREDITCARD', 
    index='Months_to_Default',
    columns='First_Reporting',
    aggfunc='nunique',  
    fill_value=0      
)
pivot_table

In [ ]:
data2['First_Reporting'] = data2.groupby('MASK_CREDITCARD')['PERIOD'].transform('min')
data2['Default'] = data2['NB_DAYS_DELINQ'].astype(float) >= 4
data2['First_Default_Date'] = data2.groupby('MASK_CREDITCARD')['PERIOD'].transform(lambda x: x[data2['Default']].min())
data2['Is_First_Default'] = data2['PERIOD'] == data2['First_Default_Date']
data2['Months_to_Default'] = np.where(
    data2['Is_First_Default'],
    (data2['PERIOD'] - data2['First_Reporting']).dt.days // 30,
    np.nan
)
data2['First_Reporting'] = data2.groupby('MASK_CREDITCARD')['PERIOD'].transform('min').dt.year

# Group by Months_to_Default and First_Reporting and count unique MASK_CREDITCARD
grouped = data2.dropna(subset=['Months_to_Default']).groupby(['Months_to_Default', 'First_Reporting'])['MASK_CREDITCARD'].nunique().unstack(fill_value=0)

total_accounts_per_year = data2.groupby('First_Reporting')['MASK_CREDITCARD'].nunique()
normalized_grouped = grouped.divide(total_accounts_per_year, axis=1)

cumulative_normalized_grouped = normalized_grouped.cumsum()
cumulative_normalized_grouped


In [ ]:
import matplotlib.pyplot as plt

# Plotting the cumulative sum for each year
plt.figure(figsize=(8, 6))

for year in cumulative_normalized_grouped.columns[1:] : 
    plt.plot(
        cumulative_normalized_grouped.index,  # Months on x-axis
        cumulative_normalized_grouped[year],  # Cumulative sum on y-axis
        label=f'Year {year}'  # Label for each year
    )

# Adding titles and labels
plt.title('Cumulative Default Rates Over Time')
plt.xlabel('Months to Default')
plt.ylabel('Cumulative Sum of Normalized Default Rates')

# Adding a legend
plt.legend(title='First Reporting Year')

# Display the plot
plt.grid(True)
plt.show()

## Bad Flag Creation

In [ ]:
#make a copy of original data
data3 = data.copy()
# data3

In [ ]:
#check the no of reporting months data available for each account
data3['entry_count'] = data3.groupby('MASK_CREDITCARD')['MASK_CREDITCARD'].transform('size')
#keep accounts having more than 18 months of reporting data
data3 = data3[data3['entry_count'] > 17].drop(['entry_count'],axis=1)
#sort data according to MASK_CREDITCARD and PERIOD
data3 = data3.sort_values(['MASK_CREDITCARD','PERIOD'])

#bad flag creation : check if the particular account has defaulted in the next 12 months and assign bad flag accordingly

indexer = pd.api.indexers.FixedForwardWindowIndexer(window_size=12)
data3['max_default'] = data3.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].transform(lambda x: x.shift(-1).rolling(window=indexer,min_periods=1).max())
data3['Bad_Flag'] = data3['max_default'] >= 4
data3['Bad_Flag'].value_counts()

#first reporting of each account
data3['First_Reporting'] = data3.groupby("MASK_CREDITCARD")['PERIOD'].transform('min')
data3['MOB'] = ((data3['PERIOD'].dt.year - data3['First_Reporting'].dt.year)*12 + (data3['PERIOD'].dt.month - data3['First_Reporting'].dt.month)).astype(int)
data3[['PERIOD','MOB']]

data3_excl1 = data3[data3['MOB'] < 6]
data3['Bad_Flag'].value_counts()

data3

In [ ]:
excluded_data = data3[ (data3['MOB'] >= 6) & (data3['NB_DAYS_DELINQ'].astype(float) < 4)]
excluded_data.loc[:,'NB_DAYS_DELINQ'] = pd.to_numeric(excluded_data['NB_DAYS_DELINQ'], errors='coerce')

bscore_df = data3.groupby('PERIOD')['MASK_CREDITCARD'].nunique().reset_index()
bscore_df.columns=['PERIOD','Total no of accts']
bscore_df['MOB < 6'] = data3[data3['MOB'] < 6].groupby("PERIOD")['MASK_CREDITCARD'].nunique().reset_index()['MASK_CREDITCARD']
bscore_df = bscore_df.merge(data3_excl1[data3_excl1['NB_DAYS_DELINQ'].astype(float) >= 4].groupby("PERIOD")['MASK_CREDITCARD'].nunique().rename("DPD >= 4"),on='PERIOD', how='left')
bscore_df = bscore_df.merge(data3[ (data3['MOB'] >= 6) & (data3['NB_DAYS_DELINQ'].astype(float) < 4)].groupby("PERIOD")['MASK_CREDITCARD'].nunique().rename("Accts after exclusion"),on='PERIOD', how='left')

bscore_df = bscore_df.merge(excluded_data.groupby('PERIOD')['Bad_Flag'].sum().rename("Bad"),on='PERIOD', how='left')
bscore_df = bscore_df.merge((excluded_data.groupby('PERIOD')['Bad_Flag'].count() - excluded_data.groupby('PERIOD')['Bad_Flag'].sum()).rename("Good"),on='PERIOD', how='left')
bscore_df['Bad Rate (in %)'] = bscore_df['Bad'] * 100 / bscore_df['Accts after exclusion']
# bscore_df = bscore_df.fillna(0)
bscore_df

In [ ]:
import matplotlib.pyplot as plt

# Plotting the cumulative sum for each year
plt.figure(figsize=(8, 6))

 
plt.plot(
    bscore_df['PERIOD'],  # x-axis
    bscore_df['Bad Rate (in %)'], color='orange')  # y-axis


# Adding titles and labels
plt.title('Bad Rates Over Time')
plt.xlabel('Period')
plt.ylabel('Bad rates')

# Display the plot
plt.show()

## Variable Creation

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
i8 = pd.read_excel(r"C:\Users\shrut\OneDrive\Desktop\Training\B-Score\data\i8_cards 2.xlsx")
i1 = pd.read_excel(r"C:\Users\shrut\OneDrive\Desktop\Training\B-Score\data\i1_card_transaction 2.xlsx")

excluded_data_1 = excluded_data.copy()
excluded_data_1['PERIOD'] = pd.to_datetime(excluded_data_1['PERIOD'])
excluded_data_1['PERIOD_MONTH'] = excluded_data_1['PERIOD'].dt.month
excluded_data_1['PERIOD_YEAR'] = excluded_data_1['PERIOD'].dt.year
i1['POST_DATE_AND_TIME'] = pd.to_datetime(i1['POST_DATE_AND_TIME'])
i1['POST_MONTH'] = i1['POST_DATE_AND_TIME'].dt.month
i1['POST_YEAR'] = i1['POST_DATE_AND_TIME'].dt.year

In [ ]:
merged_data = pd.merge(excluded_data_1, i1, how='left', left_on=['MASK_CREDITCARD','PERIOD_MONTH','PERIOD_YEAR'], right_on=['MASK_CREDITCARD','POST_MONTH','POST_YEAR'])
merged_data.shape

In [ ]:
merged_data.head(10000).to_excel(r"C:\Users\shrut\OneDrive\Desktop\Training\B-Score\data\merged_data.xlsx")

In [ ]:
merged_data.columns

In [ ]:
merged_data.shape

### Variable creation - 2

In [30]:
import pandas as pd
import numpy as np

In [31]:
i1 = pd.read_excel(r"C:\Users\shrut\OneDrive\Desktop\Training\B-Score\data\i1_card_transaction 2.xlsx")
merchant_mapping = pd.read_excel(r"C:\Users\shrut\OneDrive\Desktop\Training\B-Score\data\Merchant mapping.xlsx")

In [32]:
transaction_data = i1.copy()
mapping_data = merchant_mapping.copy()

In [33]:
transaction_data = transaction_data.sort_values(['MASK_CREDITCARD','POST_DATE_AND_TIME'])

In [34]:
mapping_data = mapping_data.melt(var_name='Category', value_name='MerchantCode').dropna()
print(mapping_data)

                   Category  MerchantCode
0                  Clothing        5137.0
1                  Clothing        5611.0
2                  Clothing        5621.0
3                  Clothing        5631.0
4                  Clothing        5641.0
..                      ...           ...
690               Groceries        5441.0
691               Groceries        5451.0
692               Groceries        5462.0
693               Groceries        5499.0
774  Financial Institutions        6012.0

[205 rows x 2 columns]


In [35]:
merchant_dict = mapping_data.set_index('MerchantCode')['Category'].to_dict()
transaction_data['Category'] = transaction_data['MERCHANT_CATEGORY'].map(merchant_dict)
transaction_data.fillna({'Category' : 'Unknown Category'}, inplace=True)

In [36]:
transaction_data[['Category','MERCHANT_CATEGORY']]

,Category,MERCHANT_CATEGORY
0,Financial Institutions,6012
1,Unknown Category,6051
2,Unknown Category,6051
3,Unknown Category,6051
4,Groceries,5411
...,...,...
809750,Unknown Category,9399
809751,Financial Institutions,6012
809752,Financial Institutions,6012
809753,Financial Institutions,6012


In [39]:
#process payment
payment_data = transaction_data.copy()
payment_data['POST_DATE_AND_TIME'] = (payment_data['POST_DATE_AND_TIME'] - pd.DateOffset(months=1)) + pd.offsets.MonthEnd(0)

filtered_data = payment_data[
    ((payment_data['TRANS_DESCRIPTION'] == 'Card Payment') | (payment_data['TRANS_DESCRIPTION'] == 'Payment')) &
    (payment_data['REVERSAL_FLAG'] == 'N')
]

payment_values = filtered_data.groupby(['MASK_CREDITCARD','POST_DATE_AND_TIME']).agg(
    No_of_transactions=('AMOUNT_IN_LCY', 'size'),  # Count the number of transactions
    Sum_of_transactions=('AMOUNT_IN_LCY', 'sum')  # Sum the transaction amounts
).reset_index()

payment_values

,MASK_CREDITCARD,POST_DATE_AND_TIME,No_of_transactions,Sum_of_transactions
0,CUKTNSPPNCCCNNNC,2012-11-30,1,10000
1,CUKTNSPPNCCCNNNC,2013-01-31,1,3000
2,CUKTNSPPNCCFNNNS,2013-05-31,1,3000
3,CUKTNSPPNCCNNNNN,2012-11-30,1,8
4,CUKTNSPPNCCNNNNN,2012-12-31,3,193
...,...,...,...,...
83351,FTKUGGWWUNGPNNNT,2012-11-30,1,106
83352,FTKUGGWWUNGPNNNT,2013-01-31,1,100
83353,FTKUGGWWUNGPNNNT,2013-02-28,1,100
83354,FTKUGGWWUNGPNNNT,2013-04-30,1,141


In [40]:
#process withdrawals
withdrawal_data = transaction_data.copy()
withdrawal_data['POST_DATE_AND_TIME'] = withdrawal_data['POST_DATE_AND_TIME'] + pd.offsets.MonthEnd(0)

filtered_data = withdrawal_data[
    ((withdrawal_data['TRANS_DESCRIPTION'] == 'Cash Advance') | (withdrawal_data['TRANS_DESCRIPTION'] == 'CASH ADVANCE') | (withdrawal_data['TRANS_DESCRIPTION'] == 'WITHDRAWAL') | (withdrawal_data['TRANS_DESCRIPTION'] == 'Withdrawal')) &
    (withdrawal_data['REVERSAL_FLAG'] == 'N')
]

withdrawal_values = filtered_data.groupby(['MASK_CREDITCARD','POST_DATE_AND_TIME']).agg(
    No_of_transactions=('AMOUNT_IN_LCY', 'size'),  # Count the number of transactions
    Sum_of_transactions=('AMOUNT_IN_LCY', 'sum')  # Sum the transaction amounts
).reset_index()

withdrawal_values

,MASK_CREDITCARD,POST_DATE_AND_TIME,No_of_transactions,Sum_of_transactions
0,CUKTNSPPNCCCNNNC,2012-11-30,3,15000
1,CUKTNSPPNCCCNNNC,2012-12-31,3,11000
2,CUKTNSPPNCCKNUTW,2013-02-28,2,3618
3,CUKTNSPPNCCNNNNN,2013-01-31,1,5000
4,CUKTNSPPNCCNNNNN,2013-02-28,3,4500
...,...,...,...,...
22014,FTKUGGWWSFWGNNNF,2012-10-31,3,6000
22015,FTKUGGWWSFWGNNNF,2012-12-31,1,100
22016,FTKUGGWWSFWGNNNF,2013-02-28,1,200
22017,FTKUGGWWUFFSNNNW,2012-03-31,2,1400


In [41]:
#process late fees
latefees_data = transaction_data.copy()
latefees_data['POST_DATE_AND_TIME'] = (latefees_data['POST_DATE_AND_TIME'] - pd.DateOffset(months=1)) + pd.offsets.MonthEnd(0)

filtered_data = latefees_data[
    ((latefees_data['TRANS_DESCRIPTION'] == 'Late Payment Charges')) &
    (latefees_data['REVERSAL_FLAG'] == 'N')
]

latefees_values = filtered_data.groupby(['MASK_CREDITCARD','POST_DATE_AND_TIME']).agg(
    No_of_transactions=('AMOUNT_IN_LCY', 'size'),  # Count the number of transactions
    Sum_of_transactions=('AMOUNT_IN_LCY', 'sum')  # Sum the transaction amounts
).reset_index()

latefees_values

,MASK_CREDITCARD,POST_DATE_AND_TIME,No_of_transactions,Sum_of_transactions
0,CUKTNSPPNCCCNNNC,2012-12-31,1,100
1,CUKTNSPPNCCCNNNC,2013-01-31,1,100
2,CUKTNSPPNCCCNNNC,2013-02-28,1,100
3,CUKTNSPPNCCCNNNC,2013-04-30,1,100
4,CUKTNSPPNCCCNNNC,2013-05-31,1,100
...,...,...,...,...
35556,FTKUGGWWUFFSNNNW,2012-06-30,1,100
35557,FTKUGGWWUFFSNNNW,2012-08-31,1,100
35558,FTKUGGWWUFFSNNNW,2012-12-31,1,100
35559,FTKUGGWWUFFSNNNW,2013-03-31,1,100
